# Задание 1

В этом задании вам предстоит сделать так, чтобы TTS начал говорить другим голосом так, чтобы можно было контролировать интонации путем присвоения токена каждому слову.

**Важно!** Добавьте ссылку на свой форк с гитхаба в команду git clone

In [80]:
!pip install -U torch==1.10

     |██████████████████████████████▎ | 834.1 MB 1.3 MB/s eta 0:00:36tcmalloc: large alloc 1147494400 bytes == 0x55ac78bf4000 @  0x7f48c63f2615 0x55ac3f44c4cc 0x55ac3f52c47a 0x55ac3f44f2ed 0x55ac3f540e1d 0x55ac3f4c2e99 0x55ac3f4bd9ee 0x55ac3f450bda 0x55ac3f4c2d00 0x55ac3f4bd9ee 0x55ac3f450bda 0x55ac3f4bf737 0x55ac3f541c66 0x55ac3f4bedaf 0x55ac3f541c66 0x55ac3f4bedaf 0x55ac3f541c66 0x55ac3f4bedaf 0x55ac3f451039 0x55ac3f494409 0x55ac3f44fc52 0x55ac3f4c2c25 0x55ac3f4bd9ee 0x55ac3f450bda 0x55ac3f4bf737 0x55ac3f4bd9ee 0x55ac3f450bda 0x55ac3f4be915 0x55ac3f450afa 0x55ac3f4bec0d 0x55ac3f4bd9ee
     |████████████████████████████████| 881.9 MB 19 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.8.0
    Uninstalling torch-1.8.0:
      Successfully uninstalled torch-1.8.0


In [1]:
ls

apex/       file.zip                   input-from-train.pt  sample_data/
down_file/  input-from-train-batch.pt  output/              TTS_HW/


In [2]:
!git clone https://github.com/psolikov/TTS_HW
!cd TTS_HW && curl gdrive.sh | bash -s https://drive.google.com/file/d/1ubAsUdWqHFY3MZJZVLqzxjH6lR_hKRyz/view?usp=sharing 
!cd TTS_HW && tar -xf libri_training_data.tar.gz


Cloning into 'TTS_HW'...
remote: Enumerating objects: 281, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 281 (delta 1), reused 5 (delta 1), pack-reused 273
Receiving objects: 100% (281/281), 4.89 MiB | 4.13 MiB/s, done.
Resolving deltas: 100% (70/70), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2874  100  2874    0     0   3583      0 --:--:-- --:--:-- --:--:--  3579
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   141    0   141    0     0    555      0 --:--:-- --:--:-- --:--:--   555
tar: libri_training_data.tar.gz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
# ! curl gdrive.sh | bash -s https://drive.google.com/file/d/1XIUvH0QXcZ6Ev0DbNSPGJIdC3-h03Sx9/view?usp=sharing
!!gdown --id 1XIUvH0QXcZ6Ev0DbNSPGJIdC3-h03Sx9

In [ ]:
!unzip file.zip -d down_file/


In [5]:
!cp -r down_file/content/TTS_HW/libri_training_data/ /content/TTS_HW/libri_training_data/

## Важно!

Следующая ячейка будет устанавливать библиотеку Apex, на колабе это может занять **12-20** минут! К сожалению, без этой библиотеки обучение данной реализации сети будет длиться слишком медленно, также из этой библиотеки используется оптимайзер FusedLamb, если же использовать стандартный Adam, обучение может стать нестабильным.

Если у вас есть своя локальная GPU, рекомендую использовать docker контейнер с Nvidia GPU Cloud - `nvcr.io/nvidia/pytorch:21.06-py3`, там эта библиотека уже предустановлена.

In [2]:
try:
  import apex
except Exception:
  ! git clone https://github.com/NVIDIA/apex.git
  % cd apex
  !pip install --target=$nb_path --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
  %cd ..

In [3]:
% cd apex
! git reset --hard a651e2c
% cd ..

/content/apex
HEAD is now at a651e2c sync-free Distributed LAMB + parameter reordering (#1055)
/content


In [4]:
!pip install -r TTS_HW/requirements.txt
!pip install transformers
!pip install torchmetrics

  Using cached DLLogger-0.1.0-py3-none-any.whl


In [5]:
!cd TTS_HW && bash scripts/download_cmudict.sh
!cd TTS_HW && bash scripts/download_fastpitch.sh
!cd TTS_HW && bash scripts/download_waveglow.sh

nvidia_fastpitch_210824.pt already downloaded.
nvidia_waveglow256pyt_fp16.pt already downloaded.


### Описание задачи
Выше была склонирована оригинальная имплементация FastPitch [отсюда](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/SpeechSynthesis/FastPitch). В ней изменен лишь даталоадер - в батче помимо текста и целевой мелспектрограммы, также возвращаются токены `Prominencе`. Необходимо понять, как можно использовать эти токены и в какое место сети необходимо их добавить, чтобы затем, на инференсе, можно было контролировать интонации.

Так как мы сильно ограничены в ресурсах - обучать сеть с нуля не получится, необходимо лишь провести finetuning с оригинального чекпоинта от Nvidia, обученного на датасете LJSpeech. От вас требуется дообучить сеть на одном из голосов из датасета LibriTTS.

Все данные уже предобработаны и загружены, токены `Prominence` были извлечены с помощью [Wavelet Prosody Toolkit](https://github.com/asuni/wavelet_prosody_toolkit). Эти значения были квантизированы, в данных могут встречаться лишь значения 1, 2, 3. Эти значения извлечены для каждого слова и продублированы для каждой фонемы.

Из-за особенностей имплементации засовывать обучение в ноутбук крайне неудобно, поэтому предлагается изменить файлы исходного кода прямо открывая их в колабе (они видны в файл менеджере слева), а затем запустить обучение, выполнив ячейку ниже. Чтобы не потерять свои изменения, рекомендую, изменив файлы, просто коммитить их к себе в репозиторий прямо из колаба. Не забывайте это делать часто, так как колаб любит уничтожать вашу среду, даже если вы просто отошли попить кофе.

Уже 10 эпох обучения достаточно (около 15 минут обучения), чтобы была слышна разница. Mel loss должен опуститься где-то до значений 0.2-0.3, чтобы TTS звучал нормально. Так как данных очень мало, то если вы оставите сеть учиться надолго, скорее всего она переобучится, поэтому учить сильно дольше 20 эпох нет смысла.

In [15]:
ls /content/output

ls: cannot access '/content/output': No such file or directory


In [18]:
# !mkdir /content/output/modules_inputs/

In [39]:
!ls 

apex	   file.zip		      input-from-train.pt  sample_data
down_file  input-from-train-batch.pt  output		   TTS_HW


In [31]:
!ls /content/output/modules_inputs

input-from-train-per-module-Conv1d.pt
input-from-train-per-module-ConvAttention.pt
input-from-train-per-module-ConvNorm.pt
input-from-train-per-module-ConvReLUNorm.pt
input-from-train-per-module-Dropout.pt
input-from-train-per-module-Embedding.pt
input-from-train-per-module-FastPitch.pt
input-from-train-per-module-FFTransformer.pt
input-from-train-per-module-LayerNorm.pt
input-from-train-per-module-Linear.pt
input-from-train-per-module-LogSoftmax.pt
input-from-train-per-module-MultiHeadAttn.pt
input-from-train-per-module-PositionalEmbedding.pt
input-from-train-per-module-PositionwiseConvFF.pt
input-from-train-per-module-ReLU.pt
input-from-train-per-module-Sequential.pt
input-from-train-per-module-Softmax.pt
input-from-train-per-module-TemporalPredictor.pt
input-from-train-per-module-TransformerLayer.pt


In [ ]:
! ls /content/output/modules_inputs

In [7]:
!cd TTS_HW && bash scripts/train.sh



AMP=true, 1x5x2 (global batch size 10)

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
train.py:46: UserWarning: PyProf is unavailable
  warnings.warn('PyProf is unavailable')
DLL 2022-01-28 21:25:46.019423 - PARAMETER | output :  ./output
DLL 2022-01-28 21:25:46.019711 - PARAMETER | dataset_path :  libri_training_data
DLL 2022-01-28 21:25:46.019794 - PARAMETER | log_file :  ./output/nvlog.json
DLL 2022-01-28 21:25:46.019874 - PARAMETER | pyprof :  False
DLL 2022-01-28 21:25:46.019949 - PARAMETER | epochs :  10
DLL 2022-01-28 21:25:46.020052 - PARAMETER | epochs_per_ch

После того как обучили сеть, прогоните ее на одной заданной фразе с разными значениями `Prominence`. Для этого в файле `inference.py` измените 384 строчку и запустите ячейку ниже. С разными значениями prominence фраза "What a day" должна произноситься с разной интонацией. Если вы этого добились, значит задание выполнено.

In [8]:
# !cd TTS_HW && bash scripts/inference_example.sh

In [9]:
import IPython
IPython.display.Audio('/content/output/audio_phrase_1_64.txt/audio_0.wav')

ValueError: ignored

In [ ]:
import IPython
IPython.display.Audio('/content/output/audio_phrase_1_64.txt/audio_0.wav')

ValueError: ignored

In [ ]:
1+3

4

# Задание 2 (5 баллов)

Это хорошо, что мы научились контролировать интонацию в произносимых фразах, но что делать на инференсе, ведь у нас нет этих значений `Prominence`? Можно попробовать научиться предсказывать их из текста. Для этого можно обучить какую-нибудь LM, при этом учить с нуля также не имеет смысла.

В задании предлагается заполнить отсутствующие ячейки ниже. Необходимый для зачета F1 score на тестовой выборке должен быть около `0.75`. Не забудьте про маски!

In [ ]:
import torch
import transformers
import os
from pathlib import Path
import numpy as np
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, BertTokenizer, BertConfig, BertModel
from tqdm import tqdm
import torchmetrics

device='cuda:0'

In [ ]:
def val_to_class(val):
    if val < 1:
        return 0
    if 1 < val < 2:
        return 1
    return 2

f1_criterion = torchmetrics.F1(num_classes=3)
acc_criterion = torchmetrics.Accuracy(num_classes=3)

class PromDataset(Dataset):
    def __init__(self, tokenizer, sentences, labels, max_len):
        self.len = len(sentences)
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        #####################
        ##### YOUR CODE #####
        #####################
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'tags': torch.tensor(label, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

sentences = []
label_proms = []
label_boundaries = []
for filename in Path('TTS_HW/libri_training_data').glob('*.prom'):
    #####################
    ##### YOUR CODE #####
    #####################


tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
train_dataset = PromDataset(tokenizer, sentences[:-10], label_proms[:-10], 200)
test_dataset = PromDataset(tokenizer, sentences[-10:], label_proms[-10:], 200)

training_loader = DataLoader(train_dataset, batch_size=24, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)


model = ##### YOUR CODE #####
model.to(device)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=2e-05)
for epoch in range(10):
    print(f"EPOCH:{epoch}")
    t_bar = tqdm(training_loader)
    model.train()
    for batch in t_bar:
        optimizer.zero_grad()
        #####################
        ##### YOUR CODE #####
        #####################
        t_bar.set_description(desc=f'loss; {loss.item()}')
        optimizer.step()

    model.eval()
    with torch.inference_mode():
        accs = []
        f1_scores = []

        for batch in test_loader:
            #####################
            ##### YOUR CODE #####
            #####################

    print(f"Accuracy: {torch.mean(torch.stack(accs)).item()}")
    print(f"F1 score: {torch.mean(torch.stack(f1_scores)).item()}")

Стоит отметить, что мы обучаем сеть на очень небольшом наборе данных - если же файнтюнить на полном ТТС датасете, то точность предсказаний меток может достичь 0.95, поэтому такую сеть можно использовать при реальном инференсе для более точного предсказания интонаций.

# Задание 3 (5 баллов) (опционально)
Если по какой-то причине что-то не получилось в предыдущих заданиях, то можно попробовать подготовить модель к инференсу. Задача - заэкспортировать код модели в ONNX, и проверить, что он работает.
Подсказка - вся модель скорее всего не заэкспортится сразу, стоит пробовать делать это по модулям, а если какая-то функция не поддерживается ONNX, то ее можно выделить отдельно и заэкспортировать, например, в TorchScript. На выходе ожидается, что есть набор onnx или torchscript модулей, выполнив последовательно которые можно получить тот же выход, что и при запуске модели на PyTorch.

Задание - по желанию, если не хочется делать что-то из того, что выше!

In [52]:
!mkdir /content/output/models/

In [65]:
!cd TTS_HW && bash scripts/train.sh



AMP=true, 1x5x2 (global batch size 10)

train.py:46: UserWarning: PyProf is unavailable
  warnings.warn('PyProf is unavailable')
Traceback (most recent call last):
  File "train.py", line 52, in <module>
    import amp_C
ImportError: /usr/local/lib/python3.7/dist-packages/amp_C.cpython-37m-x86_64-linux-gnu.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_S2_
Killing subprocess 3257
Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py", line 340, in <module>
    main()
  File "/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py", line 326, in main
    sigkill_handler(signal.SIGTERM, None)  # not coming back
  File "/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py", line 301, in sigkill_handle

In [ ]:

! ls TTS_HW/output/FastPitch_checkpoint_[1].pt

'TTS_HW/output/FastPitch_checkpoint_[1].pt'


In [ ]:
!python TTS_HW/export_torchscript.py --generator-name FastPitch --generator-checkpoint TTS_HW/output/FastPitch_checkpoint_[1].pt

In [ ]:
1+2

3

In [55]:
ls /content/output/models

FastPitch_trchscript_FastPitch.onnx


In [64]:
!pip install torch==1.8

     |████████████████████████████████| 735.5 MB 14 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.8.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.8.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.8.0 which is incompatible.


In [62]:
cat /usr/local/lib/python3.7/dist-packages/torch/_sources.py

import ast
import functools
import inspect
from textwrap import dedent
from typing import Any, Optional, Tuple, List, NamedTuple
from torch._C import ErrorReport
from torch._C._jit_tree_views import SourceRangeFactory

def get_source_lines_and_file(
    obj: Any,
    error_msg: Optional[str] = None,
) -> Tuple[List[str], int, Optional[str]]:
    """
    Wrapper around inspect.getsourcelines and inspect.getsourcefile.

    Returns: (sourcelines, file_lino, filename)
    """
    filename = None  # in case getsourcefile throws
    try:
        filename = inspect.getsourcefile(obj)
        sourcelines, file_lineno = inspect.getsourcelines(obj)
    except OSError as e:
        msg = (f"Can't get source for {obj}. TorchScript requires source access in "
               "order to carry out compilation, make sure original .py files are "
               "available.")
        if error_msg:
            msg += '\n' + error_msg
        raise OSError(msg) from e

    return sourcelines, file_lineno, 